Rapport : Quantitative Structure-Activity Relationship
Equipe : 

## **1 - Réprésentation des données**

In [24]:
# Importation des librairies
import pandas as pd
import plotly.express as px
from sklearn import linear_model
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import plotly.graph_objects as go

#### **a) Analyse de l'ensemble des données**
#### Analyse des critères statistiques des attributs

In [25]:
# Lecture des données du xlsx
data_file = "QSAR_dataset.xlsx"
# Stockage des données dans un dataframe
data = pd.read_excel(data_file,index_col=0)
print(data.shape)

(154, 75)


In [26]:
# Statistiques descriptives des 74 attributs
data.describe()

,apol,ASA+,ASA-,a_count,a_donacc,a_heavy,a_hyd,a_IC,a_nC,a_nCl,...,VSA,vsa_acc,vsa_hyd,vsa_pol,vsurf_A,vsurf_R,vsurf_S,vsurf_V,Weight,zagreb
count,154.000000,152.000000,153.000000,154.000000,154.000000,153.000000,154.000000,153.000000,154.000000,154.000000,...,154.000000,154.000000,154.000000,154.000000,154.000000,1.470000e+02,141.000000,136.000000,154.000000,154.000000
mean,34.610698,105.781739,359.928668,23.909091,0.292208,18.875817,17.350649,33.912102,11.649351,3.110390,...,273.307303,8.076532,239.944812,9.086768,2.379611,-6.802721e+08,-66.497364,-2.501405,359.813016,101.350649
std,5.951534,62.391286,111.225998,4.895461,0.862625,5.596428,5.028718,9.714722,2.472152,2.954031,...,52.783753,14.721655,59.915749,15.129738,2.637952,8.247861e+09,73.647379,2.807324,132.955027,33.487395
min,17.148172,8.778115,122.917570,12.000000,0.000000,10.000000,6.000000,12.000000,6.000000,0.000000,...,140.102050,0.000000,67.651054,0.000000,0.011998,-1.000000e+11,-209.769584,-8.247237,128.174000,46.000000
25%,31.534723,70.909811,330.864750,22.000000,0.000000,17.000000,16.000000,30.541887,12.000000,0.000000,...,246.182178,0.000000,203.302167,0.000000,0.124434,2.696068e-01,-132.566487,-5.058933,291.992000,88.000000
50%,35.579689,98.659012,389.503510,22.000000,0.000000,18.000000,17.000000,31.868664,12.000000,4.000000,...,281.160615,0.000000,253.968020,0.000000,0.376156,8.136986e-01,-10.648449,-0.411358,360.881990,94.000000
75%,38.401845,139.629990,427.294460,25.750000,0.000000,19.000000,18.000000,37.087944,12.000000,6.000000,...,295.503230,13.566921,272.261230,13.566921,4.786711,9.972196e+00,-3.509363,-0.133136,410.317990,106.000000
max,52.422001,356.764860,622.904600,43.000000,4.000000,43.000000,40.000000,86.319427,20.000000,10.000000,...,432.120120,59.150364,475.687620,59.150364,7.429943,1.611555e+01,-0.338738,-0.013318,959.170960,246.000000



#### Prétraitement des données
**Traitement des données manquantes**
Nous avons décidé de procéder au traitement des données manquantes par imputation, au lieu de simplement supprimer les dites données. Les objets n'étant pas nombreux, il y a un risque d'obtenir des résultats faussés avec ce second choix.

In [27]:
# Détermination du type, du nombre et du pourcentage de valeurs manquantes par attribut
nb_m = data.isnull().sum().sort_values()
ratio_m = (data.isnull().sum()/data.shape[0]).sort_values()

In [28]:
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)

In [29]:
# Affichage de ces données
df_manquants = pd.DataFrame({'Types': data[list(manquant.index.values)].dtypes,
                       'Nb manquants': nb_m,
                      '% de manquants': ratio_m,})
# On ne se concentre que sur les attributs aux valeurs manquantes
df_notnull = df_manquants[df_manquants["Nb manquants"]>0]
print(df_notnull)

           Types  Nb manquants  % de manquants
a_IC     float64             1        0.006494
a_heavy  float64             1        0.006494
ASA-     float64             1        0.006494
ASA+     float64             2        0.012987
vsurf_R  float64             7        0.045455
vsurf_S  float64            13        0.084416
vsurf_V  float64            18        0.116883


In [30]:
# Ajouter le ratio total et le nombre total de manquants faire une petite analyse dessus

Tout d'abord, on observe que tous les attributs manquants sont numériques. De plus, on remarque que la proportion de données manquantes est différente pour chaque attribut, on n'est donc pas dans le cas du MMCA (Données manquantes de Manière Complètement Aléatoire)<sup>**1**</sup>. On considère que nos données sont dans le cas MA (Manquantes Aléatoirement), car c'est la situation la plus courante<sup>**2**</sup>. ceci nous dirige vers une imputation par régression, adaptée pour les données MA<sup>**3**</sup>.
Puisque la régression linéaire utilisée s'appui sur les données d'autres attributs numériques, nous avons décidé d'utiliser un dataframe sans l'attribut "Class", qui n'est pas un attribut numérique.

Afin de procéder à l'imputation des valeurs manquantes de ces attributs, il faut trouver ceux qui leur sont fortement corrélés. Pour éviter de se trouver dans une impasse, nous avons décidé d'ignorer, lors de l'identification des attributs corrélés à un ADM, les autres ADM.

In [31]:
# Liste des attributs avec des données manquantes
missing_attributes = ["a_IC","a_heavy","ASA-","ASA+","vsurf_R","vsurf_S","vsurf_V"]

# Création d'un Dataframe sans l'attribut "Class"
data_noclass = data.drop(columns = ["Class"])

# Fonction permettant d'obtenir la liste des attributs fortement corrélés à attribut_name 
# On recherche au minimum un attribut fortement corrélé.
# Le paramètre count_missing_attributes est un booléen permettant de considérer (ou non)
# les autres attributs manquants
def correlated_attributes(attribute_name):
    coef = 0.9
    # Dataframe contenant les attributs fortement corrélés
    data_corr = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
    
    data_corr = data_corr.drop(missing_attributes, axis=1, errors='ignore')
    while data_corr.empty:
        coef -= 0.1
        data_corr = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
        data_corr = data_corr.drop(missing_attributes, axis=1, errors='ignore')

    res_list = list(data_corr.columns)

    return res_list,coef



In [32]:
for attribute in missing_attributes:
    print("ADM : ",attribute,"\n"," liste des attributs corrélés : ",correlated_attributes(attribute)[0],"\n",
          "     coeeficient de corrélation : ",correlated_attributes(attribute)[1])

ADM :  a_IC 
  liste des attributs corrélés :  ['a_count', 'bpol'] 
      coeeficient de corrélation :  0.7000000000000001
ADM :  a_heavy 
  liste des attributs corrélés :  ['a_hyd', 'CASA-', 'chi0', 'chi1', 'DCASA', 'VAdjMa', 'VDistMa', 'vdw_vol', 'zagreb'] 
      coeeficient de corrélation :  0.9
ADM :  ASA- 
  liste des attributs corrélés :  ['DASA', 'PEOE_VSA_NEG', 'Weight'] 
      coeeficient de corrélation :  0.9
ADM :  ASA+ 
  liste des attributs corrélés :  ['a_nH'] 
      coeeficient de corrélation :  0.9
ADM :  vsurf_R 
  liste des attributs corrélés :  ['apol', 'a_nC', 'a_nH', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'mr', 'PC-', 'SlogP', 'SMR', 'std_dim2'] 
      coeeficient de corrélation :  0.10000000000000014
ADM :  vsurf_S 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C'] 
      coeeficient de corrélation :  0.40000000000000013
ADM :  vsurf_V 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C'] 
      coeeficient de corrélation :  0.40000000000000013


In [35]:

comparison_list = []
def fill_missing_data():
    for attribute in missing_attributes:
        print(attribute)
        #Sélection des attributs corrélés à celui étudié
        predictor_attributes = correlated_attributes(attribute)[0]
        print(predictor_attributes)
        # Création du Dataframe qui contiendra le résultat
        result_data = pd.DataFrame(columns=[attribute])
        # Dataframe des données sans les lignes aux valeurs manquantes dans pour la colonne étudiée
        data_noNaN = data_noclass[~data_noclass[attribute].isna()]
        # Dataframe contenant les données d'entrainement
        X = data_noNaN[predictor_attributes]
        # Datframe de l'attribut cible
        Y = data_noNaN[attribute]

        # Création des données de test et d'entrainement 
        x_train, x_test, y_train, y_test = train_test_split(X, Y)
        # Création du modèle de régression
        model = LinearRegression().fit(X = x_train,y = y_train)
        predict = model.predict(x_test)

        # Création d'un dataframe pour observer les données
        comparison_data = pd.DataFrame(list(zip(y_test,predict)))
        comparison_data.columns = ["y_test","predict"]
        fig = px.scatter(x=y_test, y=predict)
        fig.show()
        
        r2_score = model.score(X, Y)
        print("R-squared value",r2_score)
        print(metrics.mean_absolute_error(y_test, predict))

fill_missing_data()

#for attribute in missing_attributes:
    #random_data = pd.DataFrame(columns = [attribute])
    #random_data[attribute] = data_noclass[attribute]
    #predictor_attributes = correlated_attributes(attribute)

    #random_data.fillna(random_data.select_dtypes(np.number).mean(), inplace=True)

    #model = linear_model.LinearRegression()
    #model.fit(X = data_noclass[predictor_attributes], y = random_data[attribute])
    
    #Standard Error of the regression estimates is equal to std() of the errors of each estimates
    #predict = model.predict(data_noclass[predictor_attributes])
    #std_error = (predict[random_data[attribute].notnull()] - random_data.loc[random_data[attribute].notnull(), attribute]).std()
    
    #
    #random_predict = np.random.normal(size = random_data[attribute].shape[0],  loc = predict, scale = std_error)
    

    #a = random_data[random_data[attribute].isna()]
   
    

a_IC
['a_count', 'bpol']


R-squared value 0.5552531126699632
4.393798478394434
a_heavy
['a_hyd', 'CASA-', 'chi0', 'chi1', 'DCASA', 'VAdjMa', 'VDistMa', 'vdw_vol', 'zagreb']


R-squared value 0.999969949190578
0.02688577978206192
ASA-
['DASA', 'PEOE_VSA_NEG', 'Weight']


R-squared value 0.9676164213268725
19.68059945247315
ASA+
['a_nH']


R-squared value 0.8898701610720805
15.114703981960286
vsurf_R
['apol', 'a_nC', 'a_nH', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'mr', 'PC-', 'SlogP', 'SMR', 'std_dim2']


R-squared value 0.05917313421067294
2846031017.8682647
vsurf_S
['chi1v_C', 'chi1_C']


R-squared value 0.16569443344572332
53.42002372856962
vsurf_V
['chi1v_C', 'chi1_C']


R-squared value 0.1760521409000082
2.2128157970252653


In [34]:
y0,y1 = [],[]

# Observation des attributs corrélés à ceux aux données manquante
for attribute in missing_attributes:
    y0.append((correlated_attributes(attribute,True)[1])*100)
    y1.append((correlated_attributes(attribute,False)[1])*100)

fig = go.Figure(layout_title_text="Comparaison des corrélations des ADM, en tenant en compte ou non des autres ADM")
fig.add_trace(go.Histogram(histfunc="min", y=y0, x=missing_attributes, name="Avec les autres ADM",marker_color="#B784B7"))
fig.add_trace(go.Histogram(histfunc="min", y=y1, x=missing_attributes, name="Sans les autres ADM",marker_color="#E6A4B4"))
fig.show()

TypeError: correlated_attributes() takes 1 positional argument but 2 were given

On en déduit donc, d'après les résultats précédents, qu'il est plus cohérent de considérer les autres ADM lors de l'imputation par régression linéaire. Cependant, en observant la liste des attributs corrélés pour chaque ADM, on se rend compte ue pour la meilleure corrélation possible, certains ADM dépendant d'autres ADM.

ADM :  a_IC 
  liste des attributs corrélés :  ['a_count', 'a_heavy', 'bpol', 'CASA+', 'chi0', 'chi1', 'diameter', 'PC+', 'PEOE_VSA_PNEG', 'PEOE_VSA_POL', 'radius', 'std_dim1', 'TPSA', 'VDistEq', 'VDistMa', 'vdw_vol', 'VSA']
ADM :  a_heavy 
  liste des attributs corrélés :  ['a_hyd', 'CASA-', 'chi0', 'chi1', 'DCASA', 'VAdjMa', 'VDistMa', 'vdw_vol', 'zagreb']
ADM :  ASA- 
  liste des attributs corrélés :  ['DASA', 'PEOE_VSA_NEG', 'Weight']
ADM :  ASA+ 
  liste des attributs corrélés :  ['a_nH', 'CASA+', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'std_dim2', 'VAdjEq']
ADM :  vsurf_R 
  liste des attributs corrélés :  ['apol', 'ASA+', 'a_IC', 'a_nC', 'a_nH', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'mr', 'PC-', 'SlogP', 'SMR', 'std_dim2', 'vsurf_S']
ADM :  vsurf_S 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C', 'vsurf_V']
ADM :  vsurf_V 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C', 'vsurf_S']


Création d'une fonction permettant de remplir les données manquantes d'un attribut donné en paramètres, en utilisant une régression linéaire stochastique. PQ ?
Réalisée à l'aide de la référence  4.

a_IC


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

b)

c)

2 - Mesures de distance

a)

b)

3 - Choix du modèle de classification

b)

4 - Application

In [ ]:
#okokoibn
def fun():
    return 897

### **Références**

1. https://stefvanbuuren.name/fimd/sec-MCAR.html
2. https://medium.com/analytics-vidhya/different-types-of-missing-data-59c87c046bf7
3. https://www.datacamp.com/tutorial/techniques-to-handle-missing-data-values
4. https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression